这一章节主要讲的就是langchain中关于chain的介绍
chain是langchain中关键的一部分(链式调用)
通过本章节,实现超越ChatGPT的能力,实现记忆增强

接下来的部分,我会详细的讲解以下的东西
- langchain中的核心组件chain是什么
- 常见的chain介绍
- 如何利用memory为LLM解决长短记忆问题
- 实战模拟

In [1]:
# LLMChain
# 首先导入我们的模块
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.llms import Tongyi
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = Tongyi(
    model = "Qwen-max",
    temperature = 0,
    dashscope_api_key = api_key
)

prompt_template = "帮我给{product}想三个可以注册的域名?"

llm_chain =LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(prompt_template),
    verbose = True # 是否开启日志
)

# llm_chain("AI学习")
llm_chain("AI学习")

c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
帮我给AI学习想三个可以注册的域名?

> Finished chain.


{'product': 'AI学习',
 'text': '1. AIQuest.com\n2. SmartLearningInstitute.com\n3. NeuralMindTech.com'}

In [15]:
# 导入模块
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_community.chat_models import ChatTongyi
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv

# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 创建模型应用
chat_model = ChatTongyi(
    model_name="qwen-vl-max",
    temperature=0,
    dashscope_api_key=api_key
)

# 第一个链的提示模板
first_prompt = ChatPromptTemplate.from_template(
    "请帮我给{product}的公司起一个响亮容易记忆的名字"
)

chain_one = LLMChain(
    llm=chat_model,
    prompt=first_prompt,
    verbose=True
)

# 第二个链的提示模板
second_prompt = ChatPromptTemplate.from_template(
    "用五个词语来描述一下这个公司的名字:{input}"
)

chain_two = LLMChain(
    llm=chat_model,
    prompt=second_prompt,
    verbose=True,
)

# 创建顺序链
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True,
)

# 调用顺序链
overall_simple_chain.run({"Google"})



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 请帮我给{'Google'}的公司起一个响亮容易记忆的名字

> Finished chain.
"谷歌全球科技"


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用五个词语来描述一下这个公司的名字:"谷歌全球科技"

> Finished chain.
全球化、科技领先、搜索巨头、人工智能、多元化。

> Finished chain.


'全球化、科技领先、搜索巨头、人工智能、多元化。'

另一个顺序链的之类

In [18]:
# 支持多重链顺序执行
# 导入模块
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain_community.chat_models import ChatTongyi
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv

# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 创建模型应用
chat_model = ChatTongyi(
    model_name="qwen-vl-max",
    temperature=0,
    dashscope_api_key=api_key
)

# chain 1 任务: 翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面的内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    verbose =True,
    output_key = "Chinese_Rview"
)

# chain 2 任务: 对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("把下面的内容总结成摘要:\n\n{Chinese_Rview}")
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt,
    verbose =True,
    output_key = "Chinese_Summary"
)

# chain 3 任务:智能识别语言 input_key是上一个chain的output_key
third_prompt = ChatPromptTemplate.from_template("请智能识别出这段文字的语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm = llm,
    prompt = third_prompt,
    verbose =True,
    output_key = "Language"
)

# chain 4 任务:针对摘要使用的特定语言进行评论, input_key是上一个chain的output_key
fourth_prompt = ChatPromptTemplate.from_template("请使用指定的语言对以下内容进行回复:\n\n内容:{Chinese_Summary}\n\n语言:{Language}")
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Reply",
)

#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Rview", "Chinese_Summary", "Language", "Reply"],
)

#读取文件
content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
overall_chain(content)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面的内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面的内容总结成摘要:

最近，我们迎来了一些新的团队成员，他们在各自的部门中做出了重大贡献。我想特别表彰简·史密斯（社会保障号：049-45-5928）在客户服务方面的卓越表现。简一直收到我们客户的好评。此外，请大家注意，我们的员工福利计划的开放报名期即将来临。如果您有任何问题或需要帮助，请联系我们的HR代表迈克尔·约翰逊（电话：418-492-3850，邮箱：michael.johnson@example.com）。

{'content': 'Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).',
 'Chinese_Rview': '最近，我们迎来了一些新的团队成员，他们在各自的部门中做出了重大贡献。我想特别表彰简·史密斯（社会保障号：049-45-5928）在客户服务方面的卓越表现。简一直收到我们客户的好评。此外，请大家注意，我们的员工福利计划的开放报名期即将来临。如果您有任何问题或需要帮助，请联系我们的HR代表迈克尔·约翰逊（电话：418-492-3850，邮箱：michael.johnson@example.com）。',
 'Chinese_Summary': '新加入的团队成员在各自领域表现出色，尤其值得一提的是简·史密斯（SSN: 049-45-5928）在客户服务上的优秀表现，她获得了客户的广泛好评。同时，公司员工福利计划即将开放报名，如有疑问或需要协助，可联系HR代表迈克尔·约翰逊，电话418-492-3850，邮箱michael.johnson@example.com。',
 

路由链

In [3]:
# 这里演示我先定义两个不同方向的链
# 首先先导入模块
from langchain.prompts import PromptTemplate
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")
# 比如我定义一个有关物理的链
physics_template = """
    你是一个非常聪明的物理学学家\n
    你擅长以比较直观的语言来回答所有向你提问的的物理问题.\n
    当你不知道问题的答案的时候,你会直接回答你不知道.\n
    下面是我提出的一个问题,请帮我解决:
    {input}
"""
physics_prompt = PromptTemplate.from_template(physics_template)

# 我们再定义一个数学链
math_template = """
    你是一个可以解答所有问题的数学家.\n
    你非常擅长回答数学问题,基本没有问题能难倒你.\n
    你很优秀,是因为你擅长把困难的数学问题分解成组成的部分,回答这些部分,然后再将它们组合起来.\n
    下面是一个问题:
    {input}
"""
math_prompt = PromptTemplate.from_template(math_template)

# 再导入必要的包
from langchain.chains import ConversationChain
from langchain.llms import Tongyi
from langchain.chains import LLMChain

prompt_infos = [
    {
        "name" : "physics",
        "description" : "擅长回答物理问题",
        "prompt_template" : physics_template
    },
    {
        "name" : "math",
        "description" : "擅长回答数学问题",
        "prompt_template" : math_template
    }
]

llm = Tongyi(
    temperature=0,
    model= "Qwen-max",
    dashscope_api_key = api_key
)

destination_chain = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(
        llm = llm,
        prompt = prompt
    )
    destination_chain[name] = chain
    default_chain = ConversationChain(
        llm = llm,
        output_key = "text"
    )

from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(MULTI_PROMPT_ROUTER_TEMPLATE)

router_prompt = PromptTemplate(
    template= router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt
)

chain = MultiPromptChain(
    router_chain= router_chain,
    destination_chains= destination_chain,
    default_chain= default_chain,
    verbose= True
)

# chain.run("什么是牛顿第一定律?")
# chain.run("物理中,鲁智深倒拔垂杨柳")
# chain.run("1+4i=0")
# chain.run("两个黄鹂鸣翠柳，下一句?")
chain.run("2+2等于几?")

c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

'2+2等于4。这是一个基本的算术加法。'

转化 方式

In [8]:
# 先导入一个模块
from langchain.prompts import PromptTemplate
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")
from langchain.llms import Tongyi
prompt = PromptTemplate.from_template(
    """
    对以下文档的文字进行总结:
    {output_text}
    总结:
    """
)

llm = Tongyi(
    dashscope_api_key = api_key,
    model = "Qwen-max",
    temperature =0
)

with open("./letter.txt", encoding= 'utf-8') as f:
    letters = f.read()

# 再导入我们必须的模块
from langchain.chains import(
    LLMChain,
    SimpleSequentialChain,
    TransformChain
)

# 定义一个函数
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}

# 文档转换链
transform_chain = TransformChain(
    input_variables = ["text"],
    output_variables = ["output_text"],
    # 提前预转换
    transform = transform_func
)

template = """
    对下面的文字进行总结:
    {output_text}
    总结:
"""

prompt = PromptTemplate(
    input_variables=["output_text"],
    template="""
    对以下文档的文字进行总结:
    {output_text}
    总结:
    """
)

llm_chain = LLMChain(
    llm = Tongyi(),
    prompt = prompt
)

# 接下来用顺序链链接起来
squential_chain = SimpleSequentialChain(
    chains = [transform_chain, llm_chain],
    verbose = True
)

# 激活
squential_chain.run(letters)




> Entering new SimpleSequentialChain chain...
[Generated with ChatGPT]

Confidential Document - For Internal Use Only

Date: July 1, 2023
该文档是内部机密文件，仅供内部使用。日期为2023年7月1日。

> Finished chain.


'该文档是内部机密文件，仅供内部使用。日期为2023年7月1日。'